In [1]:
# Установка gensim при необходимости
from gensim.models import Word2Vec
import numpy as np

# Два предложения
sentences = [
    ["I", "like", "machine", "learning"],
    ["I", "want", "to", "eat", "soup"],
]

# Обучаем небольшую Word2Vec-модель
model = Word2Vec(
    sentences=sentences,
    vector_size=50,
    window=2,
    min_count=1,
    sg=1,          # skip-gram
    workers=1,
    epochs=100,
    seed=42
)

# Для Word2Vec один и тот же словарный токен имеет один и тот же вектор
emb_in_s1 = model.wv["I"]
emb_in_s2 = model.wv["I"]

# Проверка совпадения
print("array_equal:", np.array_equal(emb_in_s1, emb_in_s2))
print("allclose   :", np.allclose(emb_in_s1, emb_in_s2))
print("||diff||   :", np.linalg.norm(emb_in_s1 - emb_in_s2))

# Дополнительно косинусная близость (должна быть 1.0)
cos = float(np.dot(emb_in_s1, emb_in_s2) / (np.linalg.norm(emb_in_s1) * np.linalg.norm(emb_in_s2)))
print("cosine     :", cos)

assert np.allclose(emb_in_s1, emb_in_s2), "Векторы 'I' отличаются, что не ожидается для Word2Vec."
print("OK: embedding для 'I' совпадает в обоих предложениях (Word2Vec — статическая модель).")

array_equal: True
allclose   : True
||diff||   : 0.0
cosine     : 1.0
OK: embedding для 'I' совпадает в обоих предложениях (Word2Vec — статическая модель).


In [3]:
# ...existing code...
# Установка зависимостей при необходимости и импорт
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np

# Загружаем ELMo (указываем signature и output_key в конструкторе слоя)
elmo = hub.KerasLayer(
    "https://tfhub.dev/google/elmo/3",
    signature="default",
    output_key="elmo",
    trainable=False,
)

# Два предложения
s1 = "I like machine learning"
s2 = "I want to eat soup"

# Получаем ELMo-эмбеддинги (batch=1, seq_len, 1024)
emb1 = elmo(tf.constant([s1]))[0].numpy()
emb2 = elmo(tf.constant([s2]))[0].numpy()

# Простейшая токенизация по пробелам, чтобы найти позицию "I"
tokens1 = s1.split()
tokens2 = s2.split()
idx1 = tokens1.index("I")
idx2 = tokens2.index("I")

v1 = emb1[idx1]
v2 = emb2[idx2]

# Сравнение
print("array_equal:", np.array_equal(v1, v2))
print("allclose   :", np.allclose(v1, v2))
print("||diff||   :", np.linalg.norm(v1 - v2))
cos = float(np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2)))
print("cosine     :", cos)

assert not np.allclose(v1, v2), "Ожидалось, что ELMo даст разные (контекстные) векторы для 'I' в разных предложениях."
print("OK: ELMo даёт разные контекстные embedding для 'I' в этих предложениях.")
# ...existing code...

array_equal: False
allclose   : False
||diff||   : 6.0129676
cosine     : 0.9002635478973389
OK: ELMo даёт разные контекстные embedding для 'I' в этих предложениях.
